In [ ]:
from kgx import Transformer, NeoTransformer

In [ ]:
# Monarch-Lite

# Credentials can be found from 'Registry of Biolink-compatible Neo4 instances' spreadsheet
monarch_host = ''
ports = {'bolt': 7687}
monarch_username = ''
monarch_password = ''

# Initialize NeoTransformer
monarch_lite_transformer = NeoTransformer(host=monarch_host, ports=ports, username=monarch_username, password=monarch_password)

# Define filters
monarch_lite_transformer.set_filter("subject_category", "gene")
monarch_lite_transformer.set_filter("object_category", "phenotypic_feature")
monarch_lite_transformer.set_filter("edge_label", "has_phenotype")

# Load nodes and edges from remote Monarch-Lite Neo4j instance
start = 0
end = 20000
monarch_lite_transformer.load(start=start, end=end)
monarch_lite_transformer.report()

In [ ]:
# RTX

# Credentials can be found from 'Registry of Biolink-compatible Neo4 instances' spreadsheet
rtx_host = ''
ports = {'bolt': 7687}
rtx_username = ''
rtx_password = ''

# Initialize NeoTransformer
rtx_transformer = NeoTransformer(host=rtx_uri, ports=ports, username=rtx_username, password=rtx_password)

# Define filters
rtx_transformer.set_filter("subject_category", "phenotypic_feature")
rtx_transformer.set_filter("object_category", "anatomical_entity")
rtx_transformer.set_filter("edge_label", "affects")

# Load nodes and edges from remote SemMedDB Neo4j instance
start = 0
end = 20000
rtx_transformer.load(start=start, end=end)
rtx_transformer.report()

In [ ]:
# Then we merge Monarch-Lite graph with RTX graph
graphs = [monarch_lite_transformer.graph, rtx_transformer.graph]
merged_transformer = Transformer()
merged_transformer.merge(graphs)

In [ ]:
# Now, we get a phenotypic_entity common in both the graphs
short_ear_node = [n for n in merged_transformer.graph.nodes(data=True) if n[1]['id'] == 'HP:0400005'][0]
short_ear_node

In [ ]:
# we get all in-coming edges to this node
in_edges = merged_transformer.graph.in_edges(short_ear_node[0], data=True)

# we can see that there are 8 incoming edges from gene nodes (from Monarch-Lite)
in_edges

In [ ]:
# we get all out-going edges from this node
out_edges = merged_transformer.graph.out_edges(short_ear_node[0], data=True)

# we can see that there is 1 outgoing edge from our node to anatomical_entity (from RTX)
out_edges